In [5]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.tools.giga_tool import giga_tool

from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState, START
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from pathlib import Path
import time
import datetime
import warnings
import os
import re

In [2]:
warnings.filterwarnings("ignore")

In [3]:
os.environ["GIGACHAT_CREDENTIALS"] = "Yjc1YWZhNTItMzYwYS00NmU4LTk4YjctZjU4YzAwMDIyMGJmOjJhNWE3YmVhLTAyYzctNGJhNy05NWE3LWEzY2YwNGQzYzZiNw=="

In [4]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False,
    streaming=False,
    max_tokens=8192,
    temperature=0.3,
    repetition_penalty=1.01,
    timeout=60
)

In [6]:
example_docs = [
    "Погода в МСК пасмурная",
    "Слон купил велосипед",
    "Bombini Gussini la brateelo Bombordiro Crocodilo"
]

In [7]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
docs = [Document(page_content=text) for text in example_docs]
vectorstore = FAISS.from_documents(docs, embedding)
retriever = vectorstore.as_retriever()

In [8]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [34]:
class RagTool:
    def __init__(self):
        llm = GigaChat(
            model="GigaChat-2-Max",
            verify_ssl_certs=False,
            profanity_check=False,
            streaming=False,
            max_tokens=8192,
            temperature=0.3,
            repetition_penalty=1.01,
            timeout=60
        )
        example_docs = [
            "Погода в МСК пасмурная",
            "Слон купил велосипед",
            "Bombini Gussini la brateelo Bombordiro Crocodilo"
        ]
        embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        docs = [Document(page_content=text) for text in example_docs]
        vectorstore = FAISS.from_documents(docs, embedding)
        retriver = vectorstore.as_retriever()

        self.rag_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever,
            return_source_documents=True
        )
    def run_tool(self, query):
        _query = {"query": query}
        result = self.rag_chain.invoke(_query)["result"]
        return result

In [35]:
rag_chain = RagTool()

In [36]:
class RagResult(BaseModel):
    status: str = Field(description="Статус исполнения RAG")
    message: str = Field(description="Сообщение о результате исполнения RAG")
    result: str = Field(description="Результат исполнения RAG")

few_shot_examples_rag = [
    {
        "request": "Сколько лет Льву Николаевичу Толстому?",
        "params": {"query": "Сколько лет Льву Николаевичу Толстому?"},
    }
]

@giga_tool(few_shot_examples=few_shot_examples_rag)
def rag_search(
    query: str = Field(description="Запрос в векторную БД для RAG")
) -> RagResult:
    """Использование поиска"""
    print(f"! rag_search with query: {query}")
    try:
        result = rag_chain.run_tool(query)
        return RagResult(status="OK", message="Ответ получен!", result=result)
    except Exception as e:
        return RagResult(status="FAIL", message=f"Не удалось запустить инструмент, ошибка: {e}", result=None)

In [37]:
rag_chain.run_tool("Что купил слон?")

'Слон купил велосипед.'

In [38]:
test_prompt_rag = """Ты - интелектуальный помощник.
Ты можешь использовать поиск для более точного ответа на вопросы.
Ответь на вопрос: Что купил слон?"""

In [39]:
config = {"configurable": {"thread_id": "thread_id4"}}
message = {
    "messages": [HumanMessage(content=test_prompt_rag)]
}
functions = [rag_search]
llm_with_functions = llm.bind_tools(functions)
agent_executor = create_react_agent(llm_with_functions, 
                                    functions,
                                    checkpointer=MemorySaver()
                                   )
result = agent_executor.invoke(message, config=config)

! rag_search with query: Что купил слон?


In [40]:
result["messages"]

[HumanMessage(content='Ты - интелектуальный помощник.\nТы можешь использовать поиск для более точного ответа на вопросы.\nОтветь на вопрос: Что купил слон?', additional_kwargs={}, response_metadata={}, id='06ed4a72-9bdf-4005-adfc-2506cd083d40'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'rag_search', 'arguments': {'query': 'Что купил слон?'}}, 'functions_state_id': 'ba944372-683b-4e15-8502-5db604f76d8d'}, response_metadata={'token_usage': {'prompt_tokens': 188, 'completion_tokens': 27, 'total_tokens': 215, 'precached_prompt_tokens': 0}, 'model_name': 'GigaChat-2-Max:2.0.28.2', 'x_headers': {'x-request-id': '04bbedee-8e1a-447e-9c77-ee055b62eb01', 'x-session-id': '147ee2cb-4d26-47dd-b6e4-65eb75087626', 'x-client-id': None}, 'finish_reason': 'function_call'}, id='04bbedee-8e1a-447e-9c77-ee055b62eb01', tool_calls=[{'name': 'rag_search', 'args': {'query': 'Что купил слон?'}, 'id': 'f7cd404f-43c3-46e1-9140-f6d312e02589', 'type': 'tool_call'}], usage_metadata={'outpu

In [9]:
rag_chain.invoke({"query": "Какая погода в МСК?"})

{'query': 'Какая погода в МСК?',
 'result': 'Погода в МСК пасмурная.',
 'source_documents': [Document(id='948019c5-aa54-41c1-878d-8b0839dc2068', metadata={}, page_content='Погода в МСК пасмурная'),
  Document(id='8df961f4-966b-4db0-aa2e-b0322144a451', metadata={}, page_content='Слон купил велосипед'),
  Document(id='3c83d564-0cc6-4bdd-98d6-04db044af5a2', metadata={}, page_content='Bombini Gussini la brateelo Bombordiro Crocodilo')]}

In [41]:
class SaveResult(BaseModel):
    status: str = Field(description="Статус сохранения файла")
    message: str = Field(description="Сообщение о результате сохранения в файл")

few_shot_examples_save = [
    {
        "request": "Можешь ли ты отправить сохранить строку 'Мама мыла раму' в файл по пути '/home/ts777/Code/agent4agents/test/test_msg.txt",
        "params": {"file_path": "/home/ts777/Code/agent4agents/test/test_msg.txt", "content": "Мама мыла раму"},
    }
]

@giga_tool(few_shot_examples=few_shot_examples_save)
def save_file(
    file_path: str = Field(description="Путь до файла, в который нужно сохранить содержимое."),
    content: str = Field(description="Информация, которую нужно сохранить в файл."),
) -> SaveResult:
    """Сохранить информацию в файл"""
    print(f"! save_file to {file_path}, content: {content}")
    try:
        match_python = re.search(r'```python\s*(.*?)\s*```', content, re.DOTALL)
        if match_python:
            content = match_python.group(1)
        with open(file_path, "w") as f:
            f.write(content)
        return SaveResult(status="OK", message="Файл успешно сохранен!")
    except Exception as e:
        return SaveResult(status="FAIL", message=f"Не удалось сохранить файл, ошибка: {e}")

In [42]:
class ReadResult(BaseModel):
    status: str = Field(description="Статус чтения файла")
    message: str = Field(description="Сообщение о результате чтения файла")
    result: str = Field(description="Содержимое файла")

few_shot_examples_read = [
    {
        "request": "Можешь ли ты порочитать содержимое файла '/home/ts777/Code/agent4agents/test/test_msg.txt",
        "params": {"file_path": "/home/ts777/Code/agent4agents/test/test_msg.txt"},
    }
]

@giga_tool(few_shot_examples=few_shot_examples_save)
def read_file(
    file_path: str = Field(description="Путь до файла, который нужно прочитать."),
) -> ReadResult:
    """Прочитать информацию из файла"""
    print(f"! read_file from {file_path}")
    try:
        with open(file_path, "r") as f:
            content = f.read()
        if file_path.endswith(".py"):
            content = (
                "```python\n"
                f"{content}"
                "\n```")
        return ReadResult(status="OK", message="Файл успешно прочитан!", result=content)
    except Exception as e:
        return ReadResult(status="FAIL", message=f"Не удалось прочитать файл, ошибка: {e}", result=None)

In [43]:
class MkdirResult(BaseModel):
    status: str = Field(description="Статус создания директории")
    message: str = Field(description="Сообщение о результате создания директории")

few_shot_examples_mkdir = [
    {
        "request": "Можешь ли ты создать директорию '/home/ts777/Code/agent4agents/test_dir",
        "params": {"path": "/home/ts777/Code/agent4agents/test/test_dir"},
    }
]

@giga_tool(few_shot_examples=few_shot_examples_mkdir)
def create_dir(
    path: str = Field(description="Путь до директории, которую нужно создать.")
) -> MkdirResult:
    """Создать директорию"""
    print(f"! create_dir {path}")
    try:
        if Path(path).exists():
            return MkdirResult(status="OK", message="Директория уже существует!")    
        Path(path).mkdir(parents=True, exist_ok=True)
        return MkdirResult(status="OK", message="Директория успешно создана!")
    except:
        return MkdirResult(status="FAIL", message="Не удалось создать директорию")

In [44]:
class PythonResult(BaseModel):
    status: str = Field(description="Статус запуска Python-кода")
    message: str = Field(description="Сообщение о результате запуска Python-кода.")

test_code = """```python
name = 'Ivan'
print(f'Hello, {name}!')
```python
"""

few_shot_examples_code = [
    {
        "request": f"Можешь ли ты запустить код {test_code}",
        "params": {"code_str": f"{test_code}"},
    }
]

@giga_tool(few_shot_examples=few_shot_examples_code)
def run_python_code(
    code_str: str = Field(description="Строка с Python-кодом, который нужно запустить.")
) -> PythonResult:
    """Сохранить информацию в файл"""
    print(f"! run_python_code {code_str}")
    try:
        match_python = re.search(r'```python\s*(.*?)\s*```', code_str, re.DOTALL)
        if match_python:
            code = match_python.group(1)
            result = exec(code)
        else:
            return PythonResult(status="FAIL", message="Не удалось выделить информацию из блока ```python[code]```.")
        return PythonResult(status="OK", message=f"Код успешно запущен! Результат: {result}")
    except Exception as e:
        return PythonResult(status="FAIL", message=f"Не удалось запустить код, ошибка: {e}")

In [46]:
with open("../sources/project_structure.md", "r") as f:
    project_structure = f.read()

with open("../sources/project_content.md", "r") as f:
    project_content = f.read()
    
with open("../tests/test_project_desc.md", "r") as f:
    project_task = f.read()

with open("../sources/project_agent_instruction.md", "r") as f:
    agent_instruction = f.read()

with open("../sources/project_agent_tools.md", "r") as f:
    agent_tools_instruction = f.read()

In [69]:
test_prompt = f"""Твоя рабочая директория - /home/ts777/TEST/. Работай только в ней.
Создай проект test_project и организуй его структуру с соответствии с этой структурой: {project_structure}.
Создай необходимые файлы, но пока оставь их пустыми.
"""

In [57]:
project_name = "test_agent_2"

In [58]:
test_prompt_coder = f""" Ты - очень опытный Senior Python Developer. Ты обладаешь отличными навыками написания кода,
а также умеешь создавать директории, сохранять содержимое в файл, читать данные из файла, писать Python-код и запускать его.
Твоя рабочая директория - /home/ts777/TEST/. Работай только в ней.
Твоя задача - создать полноценное рабочее приложение для решения задачи {project_task}. 
Проверь, создан ли проект {project_name}. Если нет, то создай проект {project_name} и организуй его структуру с соответствии с этой структурой: {project_structure}.
Прочитай описание каждого раздела {project_content}. 
Прочитай иструкцию по написанию LLM агента: {agent_instruction}.
Прочитай инструкцию по написанию инструментов для агента {agent_tools_instruction}.
Удели особое внимание агенту, подумай над тем, как его реализовать.
Выдели изменения, которые нужно внести в каждый раздел директории для того, чтобы он соответствовал поставленной задаче. 
Действуй шаг за шагом::
1. Сначала реализуй логику агента: напиши класс, который реализует агента и дай агенту необходимые инструменты.
2. Затем реализуй контракты взаимодействия. 
3. Затем реализуй API.
4. Далее протестируй полученный код, проверь, что он запускается.
ВАЖНО: обрабатывай каждый файл последовательно, не нужно писать все сразу!
"""

In [59]:
config = {"configurable": {"thread_id": "thread_id2"}}
message = {
    "messages": [HumanMessage(content=test_prompt_coder)]
}
functions = [save_file, read_file, create_dir, run_python_code]
llm_with_functions = llm.bind_tools(functions)
agent_executor = create_react_agent(llm_with_functions, 
                                    functions,
                                    checkpointer=MemorySaver()
                                   )
result = agent_executor.invoke(message, config=config)

! read_file from /home/ts777/TEST/test_agent_2/main.py
! save_file to /home/ts777/TEST/test_agent_2/services/agent_system.py, content: ```python
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.tools.giga_tool import giga_tool
from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from pathlib import Path
import time
import datetime
import warnings
import os
import re

class AgentSystem:
    def __init__(self):
        llm = GigaChat(
            model="GigaChat-2-Max",
            verify_ssl_certs=False,
            profanity_check=False,
            stream

In [52]:
print(result["messages"][1].content)

# План разработки проекта

1. Проверить наличие проекта `test_agent_1` в рабочей директории `/home/ts777/TEST/`.
2. Если проект не существует, создать его и организовать структуру проекта.
3. Реализовать каждый раздел проекта последовательно:
   - `main.py`: точка входа в приложение.
   - `api/endpoints.py`: FastAPI роутеры.
   - `models/agent.py`: Pydantic модели.
   - `services/agent_system.py`: логика агента.
4. Проверить работоспособность каждого раздела перед переходом к следующему.
5. Записать код в соответствующие файлы.

---

## Шаг 1: Проверка наличия проекта и создание структуры

### Проверка наличия проекта
```python
import os

project_path = "/home/ts777/TEST/test_agent_1"
if not os.path.exists(project_path):
    os.makedirs(project_path)
    os.makedirs(os.path.join(project_path, "api"))
    os.makedirs(os.path.join(project_path, "models"))
    os.makedirs(os.path.join(project_path, "services"))
    os.makedirs(os.path.join(project_path, "dependencies"))
```

### Создание 

In [63]:
state_modifier = """Ты бот, который умеет создавать директории и сохранять содержимое в файл, читать данные из файла, писать Python-код и запускать его.
Если ты написал Python-код, то сразу запусти его и проверь, что он запускается и корректно отрабатывает.
Если ты создаешь файл и директории не существует, создай ее и потом сохрани файл заново.
Твоя рабочая область, где ты будешь создавать директории и файлы: /home/ts777/TEST/. Работай только в ней! 
Спроси у пользователя все нужные данные перед действием."""

In [64]:
functions = [save_file, read_file, create_dir, run_python_code]
agent_executor = create_react_agent(llm_with_functions, 
                                    functions, 
                                    checkpointer=MemorySaver(),
                                    state_modifier=state_modifier)

In [65]:
def chat(agent_executor, thread_id: str):
    config = {"configurable": {"thread_id": thread_id}}

    while True:
        user_input = input("Клиент: ")
        if user_input == "":
            break
        print(f"User: {user_input}")
        resp = agent_executor.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)
        bot_answer = resp['messages'][-1].content
        print("\033[93m" + f"Bot: {bot_answer}" + "\033[0m")
        time.sleep(1) # Fix bug in Jupyter Notebook for VSCode

chat(agent_executor, "id_5")

Клиент:  Что находится в файле add_numbers.py?


User: Что находится в файле add_numbers.py?
! read_file from /home/ts777/TEST/add_numbers.py
Bot: В файле add_numbers.py содержится следующий Python-код:

```python
def add_numbers(a, b):
    return a + b
```


Клиент:  Протестируй его, он работает?


User: Протестируй его, он работает?
! read_file from /home/ts777/TEST/add_numbers.py
! run_python_code ```python
def add_numbers(a, b):
    return a + b

# Тестируем функцию
result = add_numbers(3, 5)
print(result)
```
8
Bot: Функция работает корректно. Тестовый запуск вернул результат: 8.


Клиент:  Круто, можешь изменить код так, чтобы там было не сложение а умножение?


User: Круто, можешь изменить код так, чтобы там было не сложение а умножение?
! read_file from /home/ts777/TEST/add_numbers.py
! save_file to /home/ts777/TEST/add_numbers.py, content: ```python
def multiply_numbers(a, b):
    return a * b
```
Bot: Код изменён и сохранён в файл add_numbers.py. Теперь функция выполняет умножение.


Клиент:  Верни, пожалуйста, содержимое файла add_numbers.py как было, а код с умножением сохрани в файл multyply_numbers.py


User: Верни, пожалуйста, содержимое файла add_numbers.py как было, а код с умножением сохрани в файл multyply_numbers.py
! read_file from /home/ts777/TEST/add_numbers.py
! save_file to /home/ts777/TEST/add_numbers.py, content: ```python
def add_numbers(a, b):
    return a + b
```
! save_file to /home/ts777/TEST/multiply_numbers.py, content: ```python
def multiply_numbers(a, b):
    return a * b
```
Bot: Исходное содержимое возвращено в файл add_numbers.py, а код с умножением сохранён в новый файл multiply_numbers.py.


Клиент:  
